In [7]:
using Random
using Statistics
using LinearAlgebra
using Plots
using Polynomials
using LinearAlgebra
using TensorCrossInterpolation
import TensorCrossInterpolation as TCI
using JLD2
using Random


In [8]:
function tci_oneshot(func, d, localdims, firstpivot, tol)
    BLAS.set_num_threads(4)

    for isearch in 1:100
        p = TCI.optfirstpivot(func, localdims, firstpivot) # search optimal fist pivot
        if abs(func(p)) > abs(func(firstpivot))
            firstpivot = p
        end
    end

    # execute tci2
    qtt, ranks, errors = TCI.crossinterpolate2(
        ComplexF64,
        func,
        localdims,
        [firstpivot],
        #tolerance=tol,
        maxiter=6,
        verbosity=1,
        loginterval=1,
        pivotsearch=:rook,
    )

    return qtt, errors
end


# 標準正規乱数を生成する関数
function gen_sn(M, I; anti_paths=true, mo_match=true, seed=seed)
    Random.seed!(seed)  # 乱数シードを固定
    if anti_paths
        sn = randn(M + 1, Int(I / 2))
        sn = hcat(sn, -sn)
    else
        sn = randn(M + 1, I)
    end
    if mo_match
        sn = (sn .- mean(sn)) ./ std(sn)
    end
    return sn
end

# アメリカン・プットオプションの価格を計算する関数
function price_american_put_option(j, K, S0s::Vector{Float64}, r::Float64, sigmas::Vector{Float64}, T::Float64; M::Int=50, I::Int=1000000, seed=seed)

    Random.seed!(seed)  # 乱数シードを固定
    #K = Ks[j[1]]
    sigma = sigmas[j[1]]
    S0 = S0s[j[2]]
    dt = T / M                       # タイムステップの長さ
    df = exp(-r * dt)                # 割引ファクター
    S = zeros(M + 1, I)              # 資産価格パスの行列
    S[1, :] .= S0                    # 初期資産価格を設定
    sn = gen_sn(M, I)                # 標準正規乱数の生成



    # 資産価格パスのシミュレーション
    for t in 2:M+1
        S[t, :] = S[t-1, :] .* exp.((r - 0.5 * sigma^2) * dt .+ sigma * sqrt(dt) .* sn[t, :])
    end

    h = max.(K .- S, 0.0)            # 即時行使価値
    V = copy(h)                      # オプション価値行列

    # Longstaff-Schwartzアルゴリズムの実装（逆順ループ）
    for t in M:-1:2
        discounted_V = V[t+1, :] * df   # 次期のオプション価値を現在価値に割引

        # 回帰モデルのフィッティング（次数3の多項式）
        p = fit(S[t, :], discounted_V, 3)

        # 継続価値の推定
        C = p.(S[t, :])

        # 行使戦略の決定
        V[t, :] = ifelse.(C .> h[t, :], V[t+1, :] * df, h[t, :])
    end

    C0 = df * mean(V[2, :])               # オプション価格の推定
    return C0
end


price_american_put_option (generic function with 1 method)

In [9]:
# 固定パラメータ
S0 = 100.0      # 初期資産価格
r = 0.05        # 無リスク金利
T = 1.0         # 満期（年）
M = 365          # タイムステップ数
I = 10000       # シミュレーションパス数
seed = 42
K = 100.0       # ストライクプライス

# 変化させるパラメータ
#K_range = range(70.0, 150.0, length=200)

#sigma_range = range(0.10, 0.30, length=200)
#Ks = collect(70:0.4:149.6) # 200
#S0s = collect(70:0.4:149.6) # 200
#sigmas = collect(0.10:0.001:0.299) #200

S0s = collect(90:0.3:119.7) # 200
sigmas = collect(0.15:0.001:0.249) #200
d = 2

tol = 1e-3


0.001

In [ ]:
op_(j) = price_american_put_option(j, K, S0s, r, sigmas, T, M=M, I=I)
#op_func = WrapFuncFloat(op_)

localdims = fill(100, d)
firstpivot = rand(1:100, d)

op_batch = TCI.ThreadedBatchEvaluator{Float64}(op_, localdims)

tt_op, errors = tci_oneshot(op_batch, d, localdims, firstpivot, tol)

In [ ]:
JLD2.@save "sigmas_stock0_10_4_american_bond10_tol$tol.jld2" tt_op
@show "end"